In [1]:
import sys
import os
import importlib
sys.path.insert(0, os.path.abspath("../data_model/"))

In [2]:
import pandas as pd
import numpy as np
from pydantic import ValidationError
import data_model
import enums as e
from utils import extract_base_type, add_enum_label_columns, add_list_objects, add_synthetic_records, map_zones
import datetime

In [3]:
importlib.reload(data_model)
importlib.reload(e)
from data_model import Respondent, Employee, AirPassenger, Trip, DepartingPassengerResident, DepartingPassengerVisitor, ArrivingPassengerResident, ArrivingPassengerVisitor, DepartingAirPassenger, ArrivingAirPassenger, Resident, Visitor

In [4]:
external_dir = "../data/external"
interim_dir = "../data/interim"
processed_dir = "../data/processed"

In [5]:
clean_survey_file = os.path.join(interim_dir, "survey_data_clean.csv")
output_csv_filename = os.path.join(processed_dir, "data_model_output.csv")

In [6]:
clean_df = pd.read_csv(clean_survey_file)

C:\Users\USVV724227\AppData\Local\Temp\ipykernel_45288\824925239.py:1: DtypeWarning: Columns (6,28,46,68,71,83,86,95,111,114,219,260,265,288,291,326,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_df = pd.read_csv(clean_survey_file)


### Select Variables to verify for the survey

In [7]:
respondent_variables = [field_name for field_name, field_info in Respondent.__fields__.items()]


trip_variables = [field_name for field_name, field_info in Trip.__fields__.items()]
trip_variables.append('respondentid')
#trip_variables.remove('valid_record')
trip_variables.remove('validation_error')
trip_variables.remove('validation_severity')
trip_variables.remove('validation_num_errors')

employee_variables = [field_name for field_name, field_info in Employee.__fields__.items()]
employee_variables.remove('trip')

#air_passenger_variables = [field_name for field_name, field_info in AirPassenger.__fields__.items()]
#air_passenger_variables.remove('trip')

air_passenger_departing_resident_variables = [field_name for field_name, field_info in DepartingPassengerResident.__fields__.items()] 
air_passenger_departing_visitor_variables = [field_name for field_name, field_info in DepartingPassengerVisitor.__fields__.items()]
air_passenger_arriving_resident_variables = [field_name for field_name, field_info in ArrivingPassengerResident.__fields__.items()]
air_passenger_arriving_visitor_variables = [field_name for field_name, field_info in ArrivingPassengerVisitor.__fields__.items()]
air_passenger_variables = list(set(air_passenger_departing_resident_variables + air_passenger_departing_visitor_variables + air_passenger_arriving_resident_variables +air_passenger_arriving_visitor_variables))

variables_to_verify = list(set(air_passenger_variables + respondent_variables + trip_variables + employee_variables))
variables_to_verify.remove('trip')
#variables_to_verify.remove('valid_record')
variables_to_verify.remove('validation_error')
variables_to_verify.remove('validation_severity')
variables_to_verify.remove('validation_num_errors')

working_df = clean_df.copy()
working_df = working_df[variables_to_verify].copy()
working_df = working_df.loc[working_df['marketsegment'].notna()].copy()

In [8]:
working_df.shape

(5934, 319)

### Serialize the data

In [9]:
trips_df = working_df[trip_variables].copy()
persons_df = working_df[list[set(employee_variables + respondent_variables + air_passenger_variables)]].copy()

In [10]:
# combined
respondent_list = add_list_objects(
        trips_df.to_dict(orient="records"),  #child list
        "respondentid", # child key
        persons_df.to_dict(orient="records"), # parent list
        "respondentid", # parent key
        "trip", # parent var
    )

In [11]:
len(respondent_list)

5934

In [12]:
employee_list = []
arriving_air_passenger_resident_list = []
arriving_air_passenger_visitor_list = []
departing_air_passenger_resident_list = []
departing_air_passenger_visitor_list = []
other_list = []
failed_records = []

for respondent in respondent_list:
     market_segment = respondent["marketsegment"]
     try:
        if market_segment == e.Type.EMPLOYEE:
            ev = Employee(** respondent)
            employee_list.append(ev)
        elif market_segment == e.Type.PASSENGER:
             passenger_segment= respondent["passenger_segment"]
             if passenger_segment == e.PassengerSegment.RESIDENT_ARRIVING:
                    apr = ArrivingPassengerResident(** respondent)
                    arriving_air_passenger_resident_list.append(apr)
             elif passenger_segment == e.PassengerSegment.VISITOR_ARRIVING:
                    apv = ArrivingPassengerVisitor(** respondent)
                    arriving_air_passenger_visitor_list.append(apv)
             elif passenger_segment == e.PassengerSegment.RESIDENT_DEPARTING:
                    dpr = DepartingPassengerResident(** respondent)
                    departing_air_passenger_resident_list.append(dpr)
             elif passenger_segment == e.PassengerSegment.VISITOR_DEPARTING:
                    dpv = DepartingPassengerVisitor(** respondent)
                    departing_air_passenger_visitor_list.append(dpv)
             else:
                    rv = Respondent(** respondent)
                    other_list.append(rv)

        else:
            rv = Respondent(** respondent)
            other_list.append(rv)
            
     except ValidationError as err:
            respondent['error_flag'] = 'failed'
            respondent['error_message'] = str(err)
            failed_records.append(respondent) 


failed_df = pd.DataFrame(failed_records)
failed_df.head()

c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `str` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  Expected `enum` but got `Terminal` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but

Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: general_use_transit_visitor_home
Critical check failed for variable: nights_visited
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed fo

c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: trip_start_time
Critical check failed for variable: main_mode
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode
Critical check failed for variable: home_location_latitude
Critical check failed for variable: home_location_longitude
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for variable: reverse_mode_predicted
Critical check failed for 

""


In [13]:
failed_df.shape

(0, 0)

In [14]:
#failed_df['error_message'].unique()

In [15]:
#failed_df['is_completed'].value_counts()

In [16]:
#failed_df['error_message'][0]

In [17]:
failed_df.to_csv('../data/processed/failed_records.csv', index = False)

In [18]:
len(failed_df)

0

In [19]:
print("Arriving Air Passengers Residents:", len(arriving_air_passenger_resident_list))
print("Arriving Air Passengers Visitors:", len(arriving_air_passenger_visitor_list))
print("Departing Air Passengers Residents:", len(departing_air_passenger_resident_list))
print("Departing Air Passengers Visitors:", len(departing_air_passenger_visitor_list))
print("Total Air Passengers:", len(arriving_air_passenger_resident_list) + len(arriving_air_passenger_visitor_list) + len(departing_air_passenger_resident_list) + len(departing_air_passenger_visitor_list))

Arriving Air Passengers Residents: 73
Arriving Air Passengers Visitors: 191
Departing Air Passengers Residents: 1795
Departing Air Passengers Visitors: 2804
Total Air Passengers: 4863


### Make Data

In [20]:
employee_df = pd.DataFrame([Employee.model_dump() for Employee in employee_list])       

c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `str` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `str` but got `li

In [21]:
#passenger_df = pd.DataFrame([AirPassenger.model_dump() for AirPassenger in air_passenger_list])

In [22]:
arriving_passenger_resident_df = pd.DataFrame([ArrivingPassengerResident.model_dump() for ArrivingPassengerResident in arriving_air_passenger_resident_list])
arriving_passenger_visitor_df = pd.DataFrame([ArrivingPassengerVisitor.model_dump() for ArrivingPassengerVisitor in arriving_air_passenger_visitor_list])
departing_passenger_resident_df = pd.DataFrame([DepartingPassengerResident.model_dump() for DepartingPassengerResident in departing_air_passenger_resident_list])
departing_passenger_visitor_df = pd.DataFrame([DepartingPassengerVisitor.model_dump() for DepartingPassengerVisitor in departing_air_passenger_visitor_list])

c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `enum` but got `Terminal` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  Expected `enum` but got `Terminal` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
c:\Users\USV

In [23]:
other_df = pd.DataFrame([Respondent.model_dump() for Respondent in other_list])
# other_df = add_enum_label_columns(other_df, Respondent)

c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [24]:
len(other_list)

264

In [25]:
trip_list = []
id_list = []
for record in employee_list + arriving_air_passenger_resident_list + arriving_air_passenger_visitor_list + departing_air_passenger_resident_list + departing_air_passenger_visitor_list  + other_list:
    trip_list.append(record.trip)
    id_list.append(record.respondentid)

trip_df = pd.DataFrame([Trip.model_dump() for Trip in trip_list])
id_df = pd.DataFrame(id_list, columns=["respondentid"])

trip_df = pd.concat([id_df, trip_df], axis=1)
trip_df = add_enum_label_columns(trip_df,Trip)


c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `list` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  Expected `float` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [26]:
output_df = pd.concat([employee_df, arriving_passenger_resident_df, arriving_passenger_visitor_df, departing_passenger_resident_df, departing_passenger_visitor_df , other_df], axis=0).reset_index(drop=True).drop(columns=["trip"])
output_df = pd.merge(output_df, trip_df, on="respondentid", how="left", suffixes = ('_person', '_trip'))

C:\Users\USVV724227\AppData\Local\Temp\ipykernel_45288\280131642.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([employee_df, arriving_passenger_resident_df, arriving_passenger_visitor_df, departing_passenger_resident_df, departing_passenger_visitor_df , other_df], axis=0).reset_index(drop=True).drop(columns=["trip"])


In [27]:
output_df.shape

(5934, 351)

In [28]:
#Consolidated validation check field
output_df['is_valid_record'] = (output_df['initial_etc_check'] == True) & (output_df['validation_severity_person']!='Critical') & (output_df['validation_severity_trip'] != 'Critical')

In [29]:
output_df = add_synthetic_records(output_df)

c:\projects\SANDAG\Questionarie_excel_to_word\Survey_Airport\data_model\utils.py:251: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df, synthetic_df], ignore_index=True)


In [30]:
output_df.head()

,validation_error_person,validation_severity_person,validation_num_errors_person,is_completed,is_self_administered,respondentid,is_pilot,record_type_synthetic,date_completed,time_completed,...,number_transit_vehicles_to_airport_label,access_mode_label,access_mode_grouped_label,parking_location_label,parking_cost_frequency_label,reimbursement_label,number_transit_vehicles_from_airport_label,egress_mode_label,egress_mode_grouped_label,is_valid_record
0,[],None,0,True,False,5473,False,0,2024-10-04,08:41:12,...,nan,nan,nan,EMPLOYEE_LOT_3665_ADMIRAL_BOLAND_WAY,MONTHLY,NOT_REIMBURSED,nan,nan,nan,True
1,[],None,0,True,False,5476,False,0,2024-10-04,08:40:04,...,ONE,WALK,WALK,nan,nan,NOT_REIMBURSED,nan,nan,nan,True
2,[],None,0,True,False,5489,False,0,2024-10-04,08:51:36,...,nan,nan,nan,EMPLOYEE_LOT_3665_ADMIRAL_BOLAND_WAY,OTHER_SPECIFY,NOT_REIMBURSED,nan,nan,nan,True
3,[],None,0,True,False,5558,False,0,2024-10-04,10:32:58,...,NONE,WALK,WALK,nan,nan,NOT_REIMBURSED,nan,nan,nan,True
4,[],None,0,True,False,5593,False,0,2024-10-04,11:09:46,...,ONE,WALK,WALK,nan,nan,REIMBURSED_EMPLOYER_CLIENT,nan,nan,nan,True


In [31]:
output_df =  add_enum_label_columns(output_df, Respondent)

output_df = add_enum_label_columns(output_df, AirPassenger)
output_df = add_enum_label_columns(output_df, DepartingAirPassenger)
output_df =  add_enum_label_columns(output_df, ArrivingAirPassenger)
output_df =  add_enum_label_columns(output_df, Resident)
output_df =  add_enum_label_columns(output_df, Visitor)

output_df = add_enum_label_columns(output_df, DepartingPassengerResident)
output_df =  add_enum_label_columns(output_df, ArrivingPassengerResident)

output_df = add_enum_label_columns(output_df, DepartingPassengerVisitor)
output_df =  add_enum_label_columns(output_df, ArrivingPassengerVisitor)

output_df =  add_enum_label_columns(output_df, Trip)
output_df =  add_enum_label_columns(output_df, Employee)

c:\projects\SANDAG\Questionarie_excel_to_word\Survey_Airport\data_model\utils.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enum_name_col] = df[field].map(enum_names).astype(str)
c:\projects\SANDAG\Questionarie_excel_to_word\Survey_Airport\data_model\utils.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[enum_name_col] = df[field].map(enum_names).astype(str)
c:\projects\SANDAG\Questionarie_excel_to_word\Survey_Airport\data_model\utils.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

In [32]:
# Define the list of important columns
important_columns = ['respondentid', 'is_completed', 'is_valid_record', 'date_completed','time_completed', 'is_pilot', 'is_self_administered', 'record_type_synthetic']

# Separate important columns and the rest of the columns
remaining_columns = [col for col in output_df.columns if col not in important_columns]

# Create the new column order
new_column_order = important_columns + sorted(remaining_columns)

# Reorder the DataFrame
output_df = output_df[new_column_order]

In [ ]:
output_df.index = output_df.index + 1
output_df.to_csv(output_csv_filename, index_label = 'unique_id')